<h3>Ankit Bansal Complex SQL 3 | Scenario based Interviews Question for Product companies</h3>

<h2>query data set<\h2>

In [0]:
%sql
create table entries ( 
name varchar(20),
address varchar(20),
email varchar(20),
floor int,
resources varchar(10));

insert into entries values 
('A','Bangalore','A@gmail.com',1,'CPU'),
('A','Bangalore','A1@gmail.com',1,'CPU'),
('A','Bangalore','A2@gmail.com',2,'DESKTOP'),
('B','Bangalore','B@gmail.com',2,'DESKTOP'),
('B','Bangalore','B1@gmail.com',2,'DESKTOP'),
('B','Bangalore','B2@gmail.com',1,'MONITOR')

num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql
select * from entries

name,address,email,floor,resources
A,Bangalore,A@gmail.com,1,CPU
A,Bangalore,A1@gmail.com,1,CPU
A,Bangalore,A2@gmail.com,2,DESKTOP
B,Bangalore,B@gmail.com,2,DESKTOP
B,Bangalore,B1@gmail.com,2,DESKTOP
B,Bangalore,B2@gmail.com,1,MONITOR


In [0]:
%sql
with cte1 as(
  select name,floor,count(floor) as m_total_vist from entries group by name,floor
  )
  , cte2 as(
select cte1.name,cte1.floor from cte1
inner join (
  select  name,max(m_total_vist)m_total_vist  from cte1 group by name
  )a1 on a1.name=cte1.name and a1.m_total_vist=cte1.m_total_vist)
,cte3 as (
select cte2.name,cte2.floor as most_visited_floor,total_vist as total_vists from cte2
inner join (
  select name,count(floor) as total_vist from entries group by name
)a2 on a2.name=cte2.name)
select cte3.name,most_visited_floor,total_vists,used_resources from cte3 
inner join(
  SELECT name, CONCAT_WS(',', COLLECT_LIST(resources ) ) as used_resources
  FROM (
    select distinct name ,resources from  entries order by name,resources
    )a0
  GROUP BY name)a3 on a3.name=cte3.name


name,most_visited_floor,total_vists,used_resources
A,1,3,"CPU,DESKTOP"
B,2,3,"DESKTOP,MONITOR"


<h2>Solution in Pyspark</h2>

In [0]:
from pyspark.sql.functions import max,col, collect_list, concat_ws

In [0]:
df=spark.sql("select * from entries")

In [0]:
df_tot_vist=df.groupBy("name").count() #total_vist
df_tot_vist=df_tot_vist.withColumnRenamed("name", "name_df_tot_vist")
df_mst_1=df.groupBy("name","floor").count()
df_mst_2=df_mst_1.groupBy("name").agg(max("count").alias("mx_cnt"))
df_mst_2=df_mst_2.withColumnRenamed("name", "name1")
df_mst=df_mst_1.join(df_mst_2,(df_mst_1["name"]==df_mst_2["name1"]) & (df_mst_1["count"]==df_mst_2["mx_cnt"]),"inner" ).\
    select("name","floor")    # max floor vist
df_dist=df.select("name","resources").distinct().orderBy("name","resources")  # get distinct value
df_used=df_dist.groupBy("name").agg(concat_ws(",", collect_list(col("resources"))).alias("used_resources")) #used_resources
df1=df_tot_vist.join(df_mst,df_mst.name==df_tot_vist.name_df_tot_vist,"inner")
df2=df1.select(df1["name"].alias("df1_name"),df1["floor"].alias("most_floor_vist"),df1["count"].alias("total_vist"))
df3=df2.join(df_used,df2.df1_name==df_used.name,"inner")
df_final=df3.select("name","most_floor_vist","total_vist","used_resources")
df_final.show()

+----+---------------+----------+---------------+
|name|most_floor_vist|total_vist| used_resources|
+----+---------------+----------+---------------+
|   A|              1|         3|    CPU,DESKTOP|
|   B|              2|         3|DESKTOP,MONITOR|
+----+---------------+----------+---------------+

